In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
def normalize(df):
    
    for column in ['x', 'y', 'z']:
        df[column] = (df[column] - df[column].mean()) / df[column].std()
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

def segment(df, window_size, step_size):
    segments = []
    for start in range(0, len(df) - window_size + 1, step_size):
        end = start + window_size
        segment = df.iloc[start:end]
        segments.append(segment)
    return segments

In [3]:
def rollingWindow(window_size=80, step_size=20):
    activities_path = 'Activity'
    activity_segments = {}

    # Iterate over each activity's folder
    for activity_name in os.listdir(activities_path):
        print(activity_name)
        activity_folder = os.path.join(activities_path, activity_name)
        if os.path.isdir(activity_folder):
            # Store segments for each activity
            activity_segments[activity_name] = []
            
            # Iterate over each CSV file within the activity's folder
            for filename in os.listdir(activity_folder):
                if filename.endswith('.csv'):
                    file_path = os.path.join(activity_folder, filename)
                    
                    # Read  CSV file
                    df = pd.read_csv(file_path)
                    
                    # Normalize data
                    df_normalized = normalize(df)
                    
                    # Segment the data using a rolling window
                    segments = segment(df_normalized, window_size, step_size)
                    
                    # Append the segments to the activity's list
                    activity_segments[activity_name].extend(segments)

    return activity_segments

def featuresAndLabels(activity_segments, len):
    X = []  # Features
    y = []  # Labels
    activity_to_label = {activity: i for i, activity in enumerate(activity_segments.keys())}

    for activity, segments in activity_segments.items():
        for segment in segments:
            if segment.shape == (len, 3):  # Adjust based off how many sec
                X.append(segment)
            else:
                print(f"Segment with shape {segment.shape} does not match the expected shape and will be skipped.")
            # Append the label
            y.append(activity_to_label[activity])

    # Convert lists to NumPy arrays
    X = np.array(X, dtype=object)  
    y = np.array(y)
    return X, y

def getModel(activity_segments, len):
    # num_classes = len(activity_segments.keys())
    # Define the model
    model = Sequential([
        # First LSTM layer returns sequences to feed into the next LSTM layer
        LSTM(64, return_sequences=True, input_shape=(len, 3)),  # Adjust 'input_shape' as needed based on your data
        # Second LSTM layer, also returning sequences
        LSTM(64, return_sequences=True),
        # Third LSTM layer, does not need to return sequences
        LSTM(64),
        # Add Dense layers for classification
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')  # 'num_classes' should be set to the number of activities you are classifying
    ])

    # Compile the model
    model.compile(optimizer='adam',  # You can adjust the optimizer, learning rate, etc.
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # Display the model's architecture
    model.summary()
    return model


In [4]:
# len = 32*5
# fivesecactivity = rollingWindow(len, int((32*5)*.25))
# fivesecnum_classes = len(fivesecactivity.keys())
# fivesec_X, fivesec_y = featuresAndLabels(fivesecactivity, len)
# X_train_val, X_test, y_train_val, y_test = train_test_split(fivesec_X, fivesec_y, test_size=0.15, random_state=42)
# # Further split training+validation data into training and validation data
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)
# X_train_padded = pad_sequences(X_train, padding='post', maxlen=len) 
# X_val_padded = pad_sequences(X_val, padding='post', maxlen=len)
# X_test_padded = pad_sequences(X_test, maxlen=len, padding='post')
# model = getModel(fivesecactivity, 32*5)
# history = model.fit(X_train_padded, y_train,
#                     validation_data=(X_val_padded, y_val),
#                     epochs=30,  # Adjust the number of epochs based on your observation of the training and validation loss/accuracy
#                     batch_size=64)

# # Evaluate the model on the test set
# test_loss, test_acc = model.evaluate(X_test_padded, y_test)
# print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

In [5]:
window_size = 160  # 2.5 seconds
step_size = 20  # 75% overlap

activities_path = 'Activity'
activity_segments = {}

# Iterate over each activity's folder
for activity_name in os.listdir(activities_path):
    print(activity_name)
    activity_folder = os.path.join(activities_path, activity_name)
    if os.path.isdir(activity_folder):
        # Store segments for each activity
        activity_segments[activity_name] = []
        
        # Iterate over each CSV file within the activity's folder
        for filename in os.listdir(activity_folder):
            if filename.endswith('.csv'):
                file_path = os.path.join(activity_folder, filename)
                
                # Read  CSV file
                df = pd.read_csv(file_path)
                
                # Normalize data
                df_normalized = normalize(df)
                
                # Segment the data using a rolling window
                segments = segment(df_normalized, window_size, step_size)
                
                # Append the segments to the activity's list
                activity_segments[activity_name].extend(segments)



Motor Behavioral Task: (All conditions seen here)
Motor Behavioral
Training ITWT - A
digit symbol task: (SIT)
TWT_B training
Training:TWT_B
B1_T2: (WALK)
trail making task: (SIT)
Naughton Test: (WALK)
B1_T1: (WALK)
Training: TWT_A
Motor behavioral task
Training ITWT -B
TNT
TWT_A Training: (WALK)
TM Comfortable Speed: (WALK)
Naughton test
B2_T1: (WALK)
B1_TWT_A: (WALK)
TWT_A training
Training ITWT - B
Training:TWT_A
TWT_B Training: (WALK)
Naughton
B2_TWT_A: (WALK)
SOT: (STAND)
Montreal Cognitive Assessment: (SIT)
HR Recovery: (STAND or SIT)
B1_TWT_B: (WALK)
B2_T2: (WALK)
Naughton Task
TM comfortable speed
Training: TWT_B
B2_TWT_B: (WALK)


In [6]:
num_classes = len(activity_segments.keys())

In [6]:

# X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# # Split training+validation data into training and validation data
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)# Number of features in your input data
# # input_shape = window_size * 3  # For X, Y, Z axes
# # # Number of distinct activities
# # num_classes = len(activity_segments.keys())

# # model = Sequential([
# #     Flatten(input_shape=(input_shape,)),  # Adjust 'input_shape' as necessary
# #     Dense(128, activation='relu'),
# #     Dropout(0.5),
# #     Dense(64, activation='relu'),
# #     Dense(num_classes, activation='softmax')  # 'num_classes' should match the number of activities
# # ])

# # model.compile(optimizer=legacy.Adam(learning_rate=0.001),
# #               loss='sparse_categorical_crossentropy',
# #               metrics=['accuracy'])
# num_classes = len(activity_segments.keys())
# model = Sequential([
#     Flatten(input_shape=(80,3)),  # Updated input shape to match the data
#     Dense(128, activation='ELU'),
#     Dropout(0.5),
#     Dense(64, activation='ELU'),
#     Dense(num_classes, activation='softmax')  # Ensure num_classes matches the number of your activities
# ])

# model.compile(optimizer=legacy.Adam(learning_rate=0.001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.summary()
# history = model.fit(X_train, y_train,
#                     validation_data=(X_val, y_val),
#                     epochs=20,  # Adjust the number of epochs as needed
#                     batch_size=64)  # And the batch size as well

In [7]:
# num_features = 3  # For X, Y, Z axes of accelerometer data
# time_steps = 80  # Based on your window size
# num_classes = len(activity_segments.keys())
# model = Sequential([
#     LSTM(64, input_shape=(time_steps, num_features), return_sequences=True),
#     LSTM(32, return_sequences=False),
#     Dense(64, activation='relu'),
#     Dense(num_classes, activation='softmax')  # num_classes is the number of activities
# ])

# model.compile(optimizer=legacy.Adam(learning_rate=0.001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.summary()

In [12]:
X = []  # Features
y = []  # Labels
len = 96
activity_to_label = {activity: i for i, activity in enumerate(activity_segments.keys())}

for activity, segments in activity_segments.items():
    for segment in segments:
        # Ensure the segment is in the correct shape, e.g., (256, 3) for 256 timesteps and 3 features (X, Y, Z)
        if segment.shape == (len, 3):  # Adjust (256, 3) based on your actual expected segment shape
            X.append(segment)
        else:
            # Handle segments that don't match the expected shape, possibly with padding, trimming, or ignoring
            print(f"Segment with shape {segment.shape} does not match the expected shape and will be skipped.")
        # Append the label
        y.append(activity_to_label[activity])

# Convert lists to NumPy arrays
X = np.array(X, dtype=object)  
y = np.array(y)

In [14]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
# Further split training+validation data into training and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(X_train, padding='post', maxlen=len) 
X_val_padded = pad_sequences(X_val, padding='post', maxlen=len)
X_test_padded = pad_sequences(X_test, maxlen=len, padding='post')

In [15]:
# Define the model
model = Sequential([
    # First LSTM layer returns sequences to feed into the next LSTM layer
    LSTM(64, return_sequences=True, input_shape=(len, 3)), 
    LSTM(64, return_sequences=True),
    LSTM(64),
    # Add Dense layers for classification
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')  # 'num_classes' should be set to the number of activities you are classifying
])

# Compile the model
model.compile(optimizer='adam',  # You can adjust the optimizer, learning rate, etc.
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 96, 64)            17408     
                                                                 
 lstm_1 (LSTM)               (None, 96, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 34)                2210      
                                                                 
Total params: 89826 (350.88 KB)
Trainable params: 89826 (350.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
history = model.fit(X_train_padded, y_train,
                    validation_data=(X_val_padded, y_val),
                    epochs=29,  # Adjust the number of epochs based on your observation of the training and validation loss/accuracy
                    batch_size=64)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

Epoch 1/29
6590/6590 [==============================] - 956s 144ms/step - loss: 2.4706 - accuracy: 0.2009 - val_loss: 2.2277 - val_accuracy: 0.2574
Epoch 2/29
6590/6590 [==============================] - 893s 136ms/step - loss: 2.1276 - accuracy: 0.2797 - val_loss: 2.0484 - val_accuracy: 0.2961
Epoch 3/29
6590/6590 [==============================] - 889s 135ms/step - loss: 1.9797 - accuracy: 0.3181 - val_loss: 1.9419 - val_accuracy: 0.3254
Epoch 4/29
6590/6590 [==============================] - 924s 140ms/step - loss: 1.8859 - accuracy: 0.3444 - val_loss: 1.8581 - val_accuracy: 0.3495
Epoch 5/29
6590/6590 [==============================] - 930s 141ms/step - loss: 1.8172 - accuracy: 0.3641 - val_loss: 1.8053 - val_accuracy: 0.3691
Epoch 6/29
6590/6590 [==============================] - 917s 139ms/step - loss: 1.7621 - accuracy: 0.3797 - val_loss: 1.7617 - val_accuracy: 0.3826
Epoch 7/29
6590/6590 [==============================] - 929s 141ms/step - loss: 1.7172 - accuracy: 0.3921 - val_

## 64 BATCH SIZE produces the best results

In [7]:
len = 160
# fivesecactivity = rollingWindow(len, 40)
fivesecactivity = activity_segments
fivesecnum_classes = num_classes ## len(fivesecactivity.keys())
fivesec_X, fivesec_y = featuresAndLabels(fivesecactivity, len)

In [8]:

X_train_val, X_test, y_train_val, y_test = train_test_split(fivesec_X, fivesec_y, test_size=0.15, random_state=42)
# Further split training+validation data into training and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)

In [9]:

X_train_padded = pad_sequences(X_train, padding='post', maxlen=len) 
X_val_padded = pad_sequences(X_val, padding='post', maxlen=len)
X_test_padded = pad_sequences(X_test, maxlen=len, padding='post')

In [10]:
model = getModel(fivesecactivity, 32*5)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 160, 64)           17408     
                                                                 
 lstm_1 (LSTM)               (None, 160, 64)           33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 34)                2210      
                                                                 
Total params: 89826 (350.88 KB)
Trainable params: 89826 (350.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X_train_padded, y_train,
                    validation_data=(X_val_padded, y_val),
                    epochs=30,  # Adjust the number of epochs based on your observation of the training and validation loss/accuracy
                    batch_size=64)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

In [24]:
32*5

160

In [ ]:
## fast fourirer transform (fft)